Partial Directed Coherence (PDC)

In [1]:
import numpy as np
from statsmodels.tsa.api import VAR

In [2]:
def fit_mvar(data, order):
    """
    Fit a Multivariate Autoregressive (MVAR) model and return coefficients.
    """
    model = VAR(data.T)  # Transpose to time × channels
    results = model.fit(order)
    # Reshape coefficients to (n_channels, n_channels, order)
    A = np.transpose(results.coefs, (1, 2, 0))  # (n_channels, n_channels, order)
    return A

In [3]:
def compute_pdc(A, freq, fs):
    """
    Compute Partial Directed Coherence (PDC) for a given frequency.

    Parameters:
    - A: MVAR coefficient matrix (n_channels × n_channels × order)
    - freq: Frequency (in Hz)
    - fs: Sampling frequency (in Hz)

    Returns:
    - PDC matrix (n_channels × n_channels)
    """
    n_channels, _, order = A.shape

    pdc_matrix = np.zeros((n_channels, n_channels, len(freq)))

    for f_idx, f in enumerate(freq):
        omega = 2 * np.pi * f/fs
        A_f = np.eye(n_channels, dtype=np.complex128)
        for k in range(order):
            A_f -= A[:, :, k] * np.exp(-1j * omega * (k + 1))

        for i in range(n_channels):
            denominator = np.sqrt(np.sum(np.abs(A_f[i, :]) ** 2))
            for j in range(n_channels):
                numerator = np.abs(A_f[i, j])
                pdc_matrix[i, j, f_idx] = numerator / denominator
                if j == i:
                    pdc_matrix[i, j, f_idx] = 0
    
    pdc_result = np.mean(pdc_matrix, axis =2)

    return pdc_result